In [46]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
Train=pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
Test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [47]:
sample_submission

,Country Name,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
0,Afghanistan,3.2253,1.1712,3.7364,0.6141,1.9883,0.0942
1,Albania,2.3565,2.3681,2.1690,1.5303,3.5665,1.7349
2,Algeria,1.0482,1.2839,1.5509,2.4556,3.0417,0.0114
3,American Samoa,3.5276,3.1392,1.1129,1.6479,3.1007,0.6840
4,Andorra,0.8961,3.9242,0.0745,2.0379,3.0965,2.1794
...,...,...,...,...,...,...,...
261,Sub-Saharan Africa,0.8879,1.0192,1.5645,1.5789,0.1476,3.6526
262,Sub-Saharan Africa (excluding high income),3.2523,1.5872,3.8270,3.4941,1.6807,1.2229
263,Sub-Saharan Africa (IDA & IBRD countries),1.7697,1.7440,1.6573,3.1732,1.3454,1.4821
264,Upper middle income,3.1682,3.6737,1.0210,2.2962,3.6815,0.6471


In [48]:
unique = Train['Country Name'].unique()

In [49]:
Train=Train.replace('..', .01020)
Test=Test.replace('..', .01020) 

In [50]:
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout

In [51]:
#Filter BY CO2 emissions (metric tons per capita)
specific_string = 'CO2 emissions (metric tons per capita)'
filtered_rows = Train[Train['Indicator'] == specific_string]
filtered_rows

,Country Code,Country Name,Indicator,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
11,Afg,Afghanistan,CO2 emissions (metric tons per capita),0.055166607,0.055292719,0.066810138,0.073004968,0.054866759,0.080361134,0.089756909,0.109940042,0.165573355,0.239506901,0.304230216,0.408965291,0.335061035,0.298087813,0.283692396,0.297971775
23,Alb,Albania,CO2 emissions (metric tons per capita),1.031567545,1.056868354,1.233001531,1.361158778,1.427944204,1.360092207,1.363988602,1.433796507,1.428249586,1.492116704,1.64276193,1.768108509,1.565921402,1.656389503,1.795712442,1.665218525
35,Alg,Algeria,CO2 emissions (metric tons per capita),2.601065339,2.520609526,2.605694613,2.751098761,2.752758782,2.85792657,2.985149131,3.023502915,3.116770947,3.187069044,3.184357,3.305232517,3.621367718,3.658468679,3.811521147,3.951960939
47,Ame,American Samoa,CO2 emissions (metric tons per capita),0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102
59,And,Andorra,CO2 emissions (metric tons per capita),7.927016355,7.725626659,7.498764979,7.238069466,7.286756008,7.206273645,6.805400082,6.890389929,7.081822365,6.995396198,7.223590934,6.957586407,6.86229282,6.674233189,6.445930663,6.485769241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,Sub,Sub-Saharan Africa,CO2 emissions (metric tons per capita),0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3155,Sub,Sub-Saharan Africa (excluding high income),CO2 emissions (metric tons per capita),0.742518227,0.787450847,0.769905273,0.793282981,0.815993983,0.798675052,0.776252936,0.77565827,0.80372919,0.755114664,0.786334412,0.763583523,0.771278211,0.790174495,0.802481827,0.763559116
3167,Sub,Sub-Saharan Africa (IDA & IBRD countries),CO2 emissions (metric tons per capita),0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3179,Upp,Upper middle income,CO2 emissions (metric tons per capita),3.167140886,3.243556127,3.347641763,3.646680369,3.955406019,4.246087066,4.524939703,4.770829501,4.86797795,4.944131035,5.315663058,5.673918008,5.764481643,5.885587659,5.857718499,5.735859454


In [52]:
#Drop Indicator Column
filtered_rows.drop(columns=['Indicator'], inplace=True)
filtered_rows

,Country Code,Country Name,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
11,Afg,Afghanistan,0.055166607,0.055292719,0.066810138,0.073004968,0.054866759,0.080361134,0.089756909,0.109940042,0.165573355,0.239506901,0.304230216,0.408965291,0.335061035,0.298087813,0.283692396,0.297971775
23,Alb,Albania,1.031567545,1.056868354,1.233001531,1.361158778,1.427944204,1.360092207,1.363988602,1.433796507,1.428249586,1.492116704,1.64276193,1.768108509,1.565921402,1.656389503,1.795712442,1.665218525
35,Alg,Algeria,2.601065339,2.520609526,2.605694613,2.751098761,2.752758782,2.85792657,2.985149131,3.023502915,3.116770947,3.187069044,3.184357,3.305232517,3.621367718,3.658468679,3.811521147,3.951960939
47,Ame,American Samoa,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102
59,And,Andorra,7.927016355,7.725626659,7.498764979,7.238069466,7.286756008,7.206273645,6.805400082,6.890389929,7.081822365,6.995396198,7.223590934,6.957586407,6.86229282,6.674233189,6.445930663,6.485769241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,Sub,Sub-Saharan Africa,0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3155,Sub,Sub-Saharan Africa (excluding high income),0.742518227,0.787450847,0.769905273,0.793282981,0.815993983,0.798675052,0.776252936,0.77565827,0.80372919,0.755114664,0.786334412,0.763583523,0.771278211,0.790174495,0.802481827,0.763559116
3167,Sub,Sub-Saharan Africa (IDA & IBRD countries),0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3179,Upp,Upper middle income,3.167140886,3.243556127,3.347641763,3.646680369,3.955406019,4.246087066,4.524939703,4.770829501,4.86797795,4.944131035,5.315663058,5.673918008,5.764481643,5.885587659,5.857718499,5.735859454


In [53]:
#Drop Country Code Column
filtered_rows.drop(columns=['Country Code'], inplace=True)
filtered_rows

,Country Name,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
11,Afghanistan,0.055166607,0.055292719,0.066810138,0.073004968,0.054866759,0.080361134,0.089756909,0.109940042,0.165573355,0.239506901,0.304230216,0.408965291,0.335061035,0.298087813,0.283692396,0.297971775
23,Albania,1.031567545,1.056868354,1.233001531,1.361158778,1.427944204,1.360092207,1.363988602,1.433796507,1.428249586,1.492116704,1.64276193,1.768108509,1.565921402,1.656389503,1.795712442,1.665218525
35,Algeria,2.601065339,2.520609526,2.605694613,2.751098761,2.752758782,2.85792657,2.985149131,3.023502915,3.116770947,3.187069044,3.184357,3.305232517,3.621367718,3.658468679,3.811521147,3.951960939
47,American Samoa,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102,0.0102
59,Andorra,7.927016355,7.725626659,7.498764979,7.238069466,7.286756008,7.206273645,6.805400082,6.890389929,7.081822365,6.995396198,7.223590934,6.957586407,6.86229282,6.674233189,6.445930663,6.485769241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,Sub-Saharan Africa,0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3155,Sub-Saharan Africa (excluding high income),0.742518227,0.787450847,0.769905273,0.793282981,0.815993983,0.798675052,0.776252936,0.77565827,0.80372919,0.755114664,0.786334412,0.763583523,0.771278211,0.790174495,0.802481827,0.763559116
3167,Sub-Saharan Africa (IDA & IBRD countries),0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3179,Upper middle income,3.167140886,3.243556127,3.347641763,3.646680369,3.955406019,4.246087066,4.524939703,4.770829501,4.86797795,4.944131035,5.315663058,5.673918008,5.764481643,5.885587659,5.857718499,5.735859454


In [54]:
#Covert Year Column in rows
melted_df = filtered_rows.melt(id_vars='Country Name', var_name='Year', value_name='CO2_Emissions')
melted_df

,Country Name,Year,CO2_Emissions
0,Afghanistan,2000 [YR2000],0.055166607
1,Albania,2000 [YR2000],1.031567545
2,Algeria,2000 [YR2000],2.601065339
3,American Samoa,2000 [YR2000],0.0102
4,Andorra,2000 [YR2000],7.927016355
...,...,...,...
4251,Sub-Saharan Africa,2015 [YR2015],0.76398389
4252,Sub-Saharan Africa (excluding high income),2015 [YR2015],0.763559116
4253,Sub-Saharan Africa (IDA & IBRD countries),2015 [YR2015],0.76398389
4254,Upper middle income,2015 [YR2015],5.735859454


In [55]:
#Sort Row in accending order by Year and Country Name
sorted_df = melted_df.groupby('Country Name').apply(lambda x: x.sort_values('Year')).reset_index(drop=True)
sorted_df.head(20)

,Country Name,Year,CO2_Emissions
0,Afghanistan,2000 [YR2000],0.055166607
1,Afghanistan,2001 [YR2001],0.055292719
2,Afghanistan,2002 [YR2002],0.066810138
3,Afghanistan,2003 [YR2003],0.073004968
4,Afghanistan,2004 [YR2004],0.054866759
5,Afghanistan,2005 [YR2005],0.080361134
6,Afghanistan,2006 [YR2006],0.089756909
7,Afghanistan,2007 [YR2007],0.109940042
8,Afghanistan,2008 [YR2008],0.165573355
9,Afghanistan,2009 [YR2009],0.239506901


In [56]:
#All country name into unique_values
unique_values = melted_df['Country Name'].unique()
unique_values.shape

(266,)

In [57]:
#Drop all th row that have value ".."
sorted_df = sorted_df[sorted_df.ne('..').all(axis=1)]

In [58]:
sorted_df.head(15)

,Country Name,Year,CO2_Emissions
0,Afghanistan,2000 [YR2000],0.055166607
1,Afghanistan,2001 [YR2001],0.055292719
2,Afghanistan,2002 [YR2002],0.066810138
3,Afghanistan,2003 [YR2003],0.073004968
4,Afghanistan,2004 [YR2004],0.054866759
5,Afghanistan,2005 [YR2005],0.080361134
6,Afghanistan,2006 [YR2006],0.089756909
7,Afghanistan,2007 [YR2007],0.109940042
8,Afghanistan,2008 [YR2008],0.165573355
9,Afghanistan,2009 [YR2009],0.239506901


In [59]:
#forcasting Using ARIMA
for i in unique_values:
    afg = sorted_df[sorted_df['Country Name'] == i]
    
    
    print(i)
    country_data=afg['CO2_Emissions'].astype(float)
    try:
        model = ARIMA(country_data, order=(1,1,0))  # Adjust order as needed
        model_fit = model.fit()

    except:
        l=0
    forecast = model_fit.forecast(steps=16)
    row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


    new_values = [i, forecast[16], forecast[17],forecast[18], forecast[19], forecast[20],forecast[30]]
    sample_submission.loc[row_index] = new_values
    
    

Afghanistan
Albania
Algeria
American Samoa
Andorra
Angola
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas, The
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
British Virgin Islands
Brunei Darussalam
Bulgaria
Burkina Faso
Burundi
Cabo Verde
Cambodia
Cameroon
Canada
Cayman Islands
Central African Republic
Chad
Channel Islands
Chile
China
Colombia
Comoros
Congo, Dem. Rep.
Congo, Rep.
Costa Rica
Cote d'Ivoire
Croatia
Cuba
Curacao
Cyprus
Czechia
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt, Arab Rep.
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Faroe Islands
Fiji
Finland
France
French Polynesia
Gabon
Gambia, The
Georgia
Germany
Ghana
Gibraltar
Greece
Greenland
Grenada
Guam
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hong Kong SAR, China
Hungary
Iceland
India
Indonesia
Iran, Islamic Rep.
Iraq
Ireland
Isle of Man
Israel
Italy
Jamaica
Japan
Jordan
K

In [60]:
sample_submission

,Country Name,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
0,Afghanistan,0.302951,0.304688,0.305294,0.305505,0.305578,0.305618
1,Albania,1.658437,1.658085,1.658066,1.658065,1.658065,1.658065
2,Algeria,4.020972,4.054884,4.071548,4.079737,4.083761,4.087645
3,American Samoa,0.010200,0.010200,0.010200,0.010200,0.010200,0.010200
4,Andorra,6.491266,6.492024,6.492129,6.492144,6.492146,6.492146
...,...,...,...,...,...,...,...
261,Sub-Saharan Africa,0.785617,0.773583,0.780277,0.776553,0.778625,0.777886
262,Sub-Saharan Africa (excluding high income),0.785203,0.773167,0.779860,0.776138,0.778208,0.777471
263,Sub-Saharan Africa (IDA & IBRD countries),0.785617,0.773583,0.780277,0.776553,0.778625,0.777886
264,Upper middle income,5.639309,5.562812,5.502202,5.454180,5.416132,5.285133


In [61]:
sample_submission.to_csv('submission.csv', index=False)